In [62]:
!pip install transformers torch
from transformers import pipeline

In [67]:
bert_gen = pipeline("text-generation", model="bert-base-uncased")
roberta_gen = pipeline("text-generation", model="roberta-base")
bart_gen = pipeline("text-generation", model="facebook/bart-base")

bert_mask = pipeline("fill-mask", model="bert-base-uncased")
roberta_mask = pipeline("fill-mask", model="roberta-base")
bart_mask = pipeline("fill-mask", model="facebook/bart-base")

bert_qa = pipeline("question-answering", model="bert-base-uncased")
roberta_qa = pipeline("question-answering", model="roberta-base")
bart_qa = pipeline("question-answering", model="facebook/bart-base")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Device set to use cpu
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Device set to use cpu
Some weights of BartForCausalLM were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['lm_head.weight', 'model.decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if 

Task 1 : Text Generation

In [68]:
prompt = "The future of Artificial Intelligence is"

print("BERT:", bert_gen(prompt))
print("RoBERTa:", roberta_gen(prompt))
print("BART:", bart_gen(prompt, max_length=40))

BERT: [{'generated_text': 'The future of Artificial Intelligence is................................................................................................................................................................................................................................................................'}]
RoBERTa: [{'generated_text': 'The future of Artificial Intelligence is'}]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


BART: [{'generated_text': 'The future of Artificial Intelligence is HydroInstall Stat reve reveilateditle630 demolition quantitative email adjustedds NCTRankedRanked locate NCT correct reve radically NCT NCT NCTds NCTInstall NCT NCTInstallInstall NCTRanked bystanders quantitativeInstallanting NCTRanked democratic democratic pictdsRankedRanked considerablydsoidal NCT NCTrequisites630OSSeffectiveeffective GH quantitative molecular 58 Reuters NCT Madame NCT NCT radicallyOSS democratic democraticasses630ds NCT NCT Noble deservedds NCT democratic democratic democraticeffective NCT locate Medicine NCT reve glove vas NCT devoted 138 democratic NCT NCT severityorgan incapableeffective NCT democraticassesatre NCT NCT Bailey democratic democratic exc NCTplane NCT democratic NCT democratic dimeliv reveoutherouther NCTWASHINGTON reveliv locateRanked NCT 1959 NCT democratic720 NCTasses Bailey occupying NCT NCT reveliv reve incapablelivlivliveffective exc... NCT NCT exc NCT NCT aggro NCT NCTasses lo

Task 2 : Fill Mask

In [69]:
sentence_bert = "The goal of Generative AI is to [MASK] new content."
sentence_roberta_bart = "The goal of Generative AI is to <mask> new content."

print("BERT:", bert_mask(sentence_bert))
print("RoBERTa:", roberta_mask(sentence_roberta_bart))
print("BART:", bart_mask(sentence_roberta_bart))

BERT: [{'score': 0.5396932363510132, 'token': 3443, 'token_str': 'create', 'sequence': 'the goal of generative ai is to create new content.'}, {'score': 0.15575720369815826, 'token': 9699, 'token_str': 'generate', 'sequence': 'the goal of generative ai is to generate new content.'}, {'score': 0.05405500903725624, 'token': 3965, 'token_str': 'produce', 'sequence': 'the goal of generative ai is to produce new content.'}, {'score': 0.04451530799269676, 'token': 4503, 'token_str': 'develop', 'sequence': 'the goal of generative ai is to develop new content.'}, {'score': 0.01757744885981083, 'token': 5587, 'token_str': 'add', 'sequence': 'the goal of generative ai is to add new content.'}]
RoBERTa: [{'score': 0.3711312413215637, 'token': 5368, 'token_str': ' generate', 'sequence': 'The goal of Generative AI is to generate new content.'}, {'score': 0.3677145540714264, 'token': 1045, 'token_str': ' create', 'sequence': 'The goal of Generative AI is to create new content.'}, {'score': 0.0835142

Task 3 : Question Answering

In [70]:
context = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question = "What are the risks?"

print("BERT:", bert_qa(question=question, context=context))
print("RoBERTa:", roberta_qa(question=question, context=context))
print("BART:", bart_qa(question=question, context=context))

BERT: {'score': 0.007528932299464941, 'start': 0, 'end': 60, 'answer': 'Generative AI poses significant risks such as hallucinations'}
RoBERTa: {'score': 0.012393318582326174, 'start': 32, 'end': 81, 'answer': 'risks such as hallucinations, bias, and deepfakes'}
BART: {'score': 0.05113985016942024, 'start': 72, 'end': 81, 'answer': 'deepfakes'}


| Task | Model | Classification (Success/Failure) | Observation (What actually happened?) | Why did this happen? (Architectural Reason) |
|------|-------|----------------------------------|----------------------------------------|--------------------------------------------|
| Generation | BERT | Failure | Could not generate a meaningful continuation. | BERT is an encoder-only so, it isn't trained to predict the next word |
|  | RoBERTa | Failure | The pipeline failed to generate proper text. | RoBERTa is also encoder-only and does not have a decoder for sequence generation. |
|  | BART | Success | Generated continuation of the prompt. | BART is an encoder–decoder arch used for sequence generation. |
| Fill-Mask | BERT | Success | Predicted 'create', 'generate', 'produce'. | BERT is trained on MLM. |
|  | RoBERTa | Success | Predicted accurate tokens. | RoBERTa is an optimized encoder trained on MLM. |
|  | BART | Partial Success | Predictions were less direct. | BART is trained with denoising autoencoding, not pure MLM. |
| QA | BERT | Partial Success | Extracted relevant phrase such as with low confidence. | Base BERT is not fine-tuned for question answering tasks. |
|  | RoBERTa | Partial Success | Returned similar or slightly better span selection than BERT. | Specifically trained for QA purposes. |
|  | BART | Partial Success | Sometimes produced the correct answer. | Correctly map question-context pairs, but without fine-tuning, its performance is unstable. |